# 0. Library & Problem

[MainPage](https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists)

**문제 정의**
* 누가 이직 결심을 하고 있는지?를 찾는 것 
* 미싱 데이터 채우기(여러 컬럼들 간의 연관성 찾기)

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML

# 1. Preview

In [4]:
train = pd.read_csv("C:/Users/HOME/Dropbox/dataset/used/hr-analytics-job-change-of-data-scientists/aug_train.csv")
test = pd.read_csv("C:/Users/HOME/Dropbox/dataset/used/hr-analytics-job-change-of-data-scientists/aug_test.csv")
submission = pd.read_csv("C:/Users/HOME/Dropbox/dataset/used/hr-analytics-job-change-of-data-scientists/sample_submission.csv")
print("train")
display(HTML(train.head().to_html()))
print("test")
display(HTML(test.head().to_html()))
print(f'train.shape : {train.shape}')
print(f'test.shape : {test.shape}') #test는 unseen이 기본 전제이다.

train


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


test


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72


train.shape : (19158, 14)
test.shape : (2129, 13)


* 연속적인 정보들이 별로 없다. index와 hour는 숫자형이지만 범주형적인 속성을 띄고 있다.


In [7]:
#NULL과 타입 정보
print(train.info())
print('*'*100)
print(train.isnull().sum().to_frame().sort_values(by=0, ascending=False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

* 대부분이 범주형 변수 NULL값이 다수인 변수 4개

In [9]:
#유니크 정보
for col in train:
    if len(train[col].unique()) <= 30:
        print(f'{col} : {len(train[col].unique())}, {train[col].unique()}')
    else:
        print(f'{col} : {len(train[col].unique())}')

enrollee_id : 19158
city : 123
city_development_index : 93
gender : 4, ['Male' nan 'Female' 'Other']
relevent_experience : 2, ['Has relevent experience' 'No relevent experience']
enrolled_university : 4, ['no_enrollment' 'Full time course' nan 'Part time course']
education_level : 6, ['Graduate' 'Masters' 'High School' nan 'Phd' 'Primary School']
major_discipline : 7, ['STEM' 'Business Degree' nan 'Arts' 'Humanities' 'No Major' 'Other']
experience : 23, ['>20' '15' '5' '<1' '11' '13' '7' '17' '2' '16' '1' '4' '10' '14' '18'
 '19' '12' '3' '6' '9' '8' '20' nan]
company_size : 9, [nan '50-99' '<10' '10000+' '5000-9999' '1000-4999' '10/49' '100-500'
 '500-999']
company_type : 7, [nan 'Pvt Ltd' 'Funded Startup' 'Early Stage Startup' 'Other'
 'Public Sector' 'NGO']
last_new_job : 7, ['1' '>4' 'never' '4' '3' '2' nan]
training_hours : 241
target : 2, [1. 0.]


**분석 목적**
* 어떤 사람들이 언제 왜 직장을 떠나고 싶어할까?
    * 연차가 높아서?(지루함)
    * 회사 규모?
    * 도시 지수가 높은 사람들? 교육 수준? 등등

# 1. Visualization1 - Depth1

**여러 변수들의 빈도를 나타낼 수 있는 플랏**
1. figure 객체 만들기
2. seaborn 카운트 플랏(범주형 변수)
3. histogram(연속형 변수)